In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
from torch import nn
from torch import optim

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/root/Downloads/mobile-price-classification/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/root/Downloads/mobile-price-classification/train.csv
/root/Downloads/mobile-price-classification/test.csv


In [2]:
path = '/root/Downloads/mobile-price-classification/'
train_df = pd.read_csv(path + 'train.csv')

train_size = int(0.8 * len(train_df))
test_size = len(train_df) - train_size
train_dataset = train_df[:train_size]
test_dataset = train_df[train_size:len(train_df)]

In [3]:
print(len(train_dataset))
print(len(test_dataset))

1600
400


In [4]:
batch_size = 100
n_iters = 80
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
num_epochs

5

In [5]:
y_label = 'price_range'

X_train = train_dataset.drop(y_label, axis=1)
y_train = train_dataset[y_label]
X_test = train_dataset.drop(y_label, axis=1)
y_test = train_dataset[y_label]

In [6]:
train_target = torch.tensor(y_train.values)
train = torch.tensor(X_train.values.astype(np.float32))
train_tensor = torch.utils.data.TensorDataset(train, train_target) 
train_loader = torch.utils.data.DataLoader(dataset=train_tensor, batch_size=batch_size, shuffle=True)

In [7]:
test_target = torch.tensor(y_test.values)
test = torch.tensor(X_test.values.astype(np.float32))
test_tensor = torch.utils.data.TensorDataset(test, test_target) 
test_loader = torch.utils.data.DataLoader(dataset=test_tensor, batch_size=batch_size, shuffle=False)

In [8]:
import collections
print(isinstance(train_loader, collections.Iterable))
print(isinstance(test_loader, collections.Iterable))

True
True


In [9]:
input_dim = len(train_dataset.columns) - 1
output_dim = len(train_dataset[y_label].unique())

In [10]:
# # Same as linear regression! 
# class LogisticRegressionModel(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super(LogisticRegressionModel, self).__init__()
#         self.linear = nn.Linear(input_dim, output_dim)
    
#     def forward(self, x):
#         out = self.linear(x)
#         return out

In [20]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.sigmoid = nn.Sigmoid()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)
        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)
        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

In [21]:

#model = LogisticRegressionModel(input_dim, output_dim)

hidden_dim = 100
model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [22]:
criterion = nn.CrossEntropyLoss()  

In [23]:
learning_rate = 0.0001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [25]:
print(model.parameters())

print(len(list(model.parameters())))

## FC 1 Parameters 
print(list(model.parameters())[0].size())

## FC 1 Bias Parameters
print(list(model.parameters())[1].size())

print(list(model.parameters())[2].size())

print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x7f37b54d50c0>
4
torch.Size([100, 20])
torch.Size([100])
torch.Size([4, 100])
torch.Size([4])


In [26]:
iter = 0
for epoch in range(num_epochs):
    for i, (batch, labels) in enumerate(train_loader):
        # Load images as Variable
        #batch = batch.view(-1, 20).requires_grad_()
        #batch = batch.requires_grad_()
        #labels = labels
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(batch)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 10 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for batch, labels in test_loader:
                # Load images to a Torch Variable
                #batch = batch.requires_grad_()
                
                # Forward pass only to get logits/output
                outputs = model(batch)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 10. Loss: 1.590938925743103. Accuracy: 29
Iteration: 20. Loss: 1.5047852993011475. Accuracy: 29
Iteration: 30. Loss: 1.5924991369247437. Accuracy: 29
Iteration: 40. Loss: 1.5016257762908936. Accuracy: 30
Iteration: 50. Loss: 1.5649871826171875. Accuracy: 30
Iteration: 60. Loss: 1.4837646484375. Accuracy: 30
Iteration: 70. Loss: 1.5077613592147827. Accuracy: 30
Iteration: 80. Loss: 1.4780975580215454. Accuracy: 30
